## Captação dos dados VIA API

Utilizando NewsAPI para fazer resgate das notícias dos últimos 30 dias

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Obter a data atual
data_atual = datetime.now()
data = data_atual - timedelta(days=30)
data_formatada = data.strftime("%Y-%m-%d")

# URL da API
url = (f'https://newsapi.org/v2/everything?'
       'q=agronegócio&'
       'from={data_formatada}'
       'sortBy=popularity&'
       'apiKey=df22a809929b459796eda53179209ec0')

# Fazendo a requisição
response = requests.get(url)

if response.status_code == 200:
    noticias = response.json()
    articles = noticias.get("articles", [])
    
    dados = []

    if articles:
        for artigo in articles:
            dados.append({
                "Título": artigo.get("title", "N/A"),
                "Descrição": artigo.get("description", "N/A"),
                "Data de Publicação": artigo.get("publishedAt", "N/A"),
                "URL": artigo.get("url", "N/A"),
                "Fonte": artigo.get("source", {}).get("name", "N/A")  # Fonte do artigo
            })
        
        df = pd.DataFrame(dados)
        display(df.head(10))
    else:
        print("Nenhuma notícia encontrada.")
else:
    print(f"Erro: {response.status_code}, {response.json().get('message')}")


,Título,Descrição,Data de Publicação,URL,Fonte
0,Primeiro satélite brasileiro lançado em territ...,Empresa que coloca IA no agronegócio financia ...,2024-11-15T17:48:25Z,https://olhardigital.com.br/2024/11/15/ciencia...,Olhardigital.com.br
1,[Removed],[Removed],2024-11-18T21:01:00Z,https://removed.com,[Removed]
2,Crise das carnes: agronegócio espera retrataçã...,Alexandre Bompard questionou qualidade do prod...,2024-11-25T20:51:35Z,https://veja.abril.com.br/coluna/radar-economi...,Abril.com.br
3,Bancada do agro confirma apoio a Hugo Motta à ...,O presidente da Frente Parlamentar do Agronegó...,2024-12-03T16:44:40Z,https://www.terra.com.br/noticias/brasil/polit...,Terra.com.br
4,Agronegócio: Veja as novas tarifas após acordo...,Assinado nesta sexta-feira (6) após 25 anos de...,2024-12-08T15:19:34Z,https://dinheirama.com/agronegocio-veja-as-nov...,Dinheirama.com
5,Ações do BB caem 3% após “bandeira laranja” no...,Os resultados do Banco do Brasil (BBAS3) no te...,2024-11-14T14:32:13Z,https://dinheirama.com/acoes-do-bb-caem-3-apos...,Dinheirama.com
6,Produtores de frango e suínos também cogitaram...,Rede francesa divulgou retratação nesta terça-...,2024-11-26T12:26:33Z,https://veja.abril.com.br/coluna/radar-economi...,Abril.com.br
7,Lula e Xi Jinping assinam 37 acordos de cooper...,"São mais de 15 áreas de atuação, incluindo int...",2024-11-20T18:03:01Z,https://veja.abril.com.br/mundo/lula-e-xi-jinp...,Abril.com.br
8,Fim do boicote? Como agronegócio recebeu as de...,'Nós não vamos estender o boicote porque podem...,2024-11-26T12:52:06Z,https://veja.abril.com.br/coluna/radar-economi...,Abril.com.br
9,"Agronegócio está magoado, mas celebra Brasil u...",Executivos do setor seguem com algum incômodo ...,2024-12-01T18:44:25Z,https://veja.abril.com.br/coluna/radar-economi...,Abril.com.br
